# On-Chain Data Collection
In this notebook, we take care of funding and settlement transactions and addresses. The sections are:
1. Funding Side
    - 1.1 Fetch Funding Txs 
    - 1.2 Clean Funding Txs 
    - 1.3 Funding Addresses 
    - 1.4 Store Data 
2. Settlement Side
    - 2.1 Fetch Settlement Txs 
    - 2.2 Clean Settlement Txs 
    - 2.3 Settlement Addresses 
    - 2.4 Store Data 


In [1]:
import sys
sys.path.append("..")

from utils import read_json, write_json, get_blockstream_tx, get_blockchain_page, wasabi_addresses, get_available_txs

# input files
from utils import channels_file, samourai_txs_file

# output files
from utils import funding_txs_file, funding_addresses_file, funding_addresses_csv_file, unilateral_settlement_txs_with_p2wsh_in_outputs_spenders_file, spender_details_file, settlement_txs_with_punishment_file, funded_address_settlement_txs_file, settlement_txs_file, settlement_addresses_file, settlement_addresses_csv_file

import pandas as pd

# 1. Funding Side

In this section, we fetch, clean and store funding txs and funding addresses of the channels in our dataset.

#### Inputs (made available):
- `channels_file`
- `samourai_txs_file`

#### Outputs (made available):
- `funding_txs_file`
- `funding_addresses_file`
- `funding_addresses_csv_file`

You can either run all the cells (some might take several minutes to complete, look for the \*\*WARNING** comment in the cells) or run directly the "output files" cell and have a look at the data.

In [2]:
# input files
channels_df = pd.read_csv(channels_file) # from step #0
samourai_txs = set(read_json(samourai_txs_file)) # from https://github.com/nopara73/WasabiVsSamourai/tree/master/WasabiVsSamourai

In [3]:
# output files (the results of section 1. Funding Side)
funding_txs = read_json(funding_txs_file)
funding_addresses = read_json(funding_addresses_file)
# funding_addresses_df = pd.read_csv(funding_addresses_csv_file) # if you prefer pandas

## 1.1 Fetch Funding Txs
Here you find the code to fetch the funding txs from Blockstream API, starting from a list of channel points. Not needed if you already read the output files above.

In [4]:
# initialize dict
funding_txs = dict()
for channel in channels_df.chan_point.values:
    hsh = channel.split(':')[0]
    if hsh not in funding_txs:
        funding_txs[hsh] = None
print('Number of funding txs:', len(funding_txs))

Number of funding txs: 98240


In [6]:
# (re)run until all txs are fetched
# **WARNING**, it might take some minutes
for i, hsh in enumerate(funding_txs):
    if not isinstance(funding_txs[hsh], dict):
        print('Fetching funding tx', i, end='\r')
        funding_txs[hsh] = get_blockstream_tx(hsh)

## 1.2 Clean Funding Txs
Here you find the code to:
- remove txs not in GraphSense (available till `last_block`)
- remove coinjoins (wasabi and samourai)

Not needed if you already read the output files above.

In [4]:
funding_txs = get_available_txs(funding_txs) # Remove txs not in GraphSense

In [5]:
# Remove coinjoins (wasabi and samourai)
wasabi_coinjoins = set()
for hsh, tx in funding_txs.items():
    for vout in tx['vout']:
        if vout['scriptpubkey_address'] in wasabi_addresses:
            wasabi_coinjoins.add(hsh)
            
coinjoin_free_funding_txs = dict()
for hsh, tx in funding_txs.items():
    if hsh not in wasabi_coinjoins and hsh not in samourai_txs:
        coinjoin_free_funding_txs[hsh] = tx
funding_txs = coinjoin_free_funding_txs

## 1.3 Funding Addresses
Here you find the code to compute and save the set of funding addresses.
Not needed if you already read the output files above.

In [7]:
funding_addresses = set()
for hsh, tx in funding_txs.items():
    for vin in tx['vin']:
        funding_addresses.add(vin['prevout']['scriptpubkey_address'])
funding_addresses = list(funding_addresses)

## 1.4 Store Data

In [9]:
write_json(funding_txs, funding_txs_file)
write_json(funding_addresses, funding_addresses_file)
pd.DataFrame(funding_addresses, columns='address').to_csv(funding_addresses_csv_file, index=False)

# 2. Settlement Side

In this section, we fetch, clean and store funded addresses (multisig addresses), settlement txs and settlement addresses of the channels in our dataset.

#### Inputs (made available):
- `channels_file`

#### Outputs (made available):
- `funded_address_settlement_txs_file`
- `settlement_txs_file`
- `settlement_addresses_file`

You can either run all the cells (some might take several minutes to complete, look for the \*\*WARNING** comment in the cells) or run directly the "output files" cell and have a look at the data.

In [4]:
# input files
channels_df = pd.read_csv(channels_file)

In [5]:
# output files (the results of section 2. Settlement Side)
funded_address_settlement_txs = read_json(funded_address_settlement_txs_file)
settlement_txs = read_json(settlement_txs_file)
settlement_addresses = read_json(settlement_addresses_file)
# settlement_addresses_df = pd.read_csv(settlement_addresses_csv_file) # if you prefer pandas

## 2.1 Fetch Settlement Txs
Here you find the code to fetch the settlement txs from Blockstream API, starting from a list of channel points. Not needed if you already read the output files above.

In [14]:
# initialize dict
funded_address_settlement_txs = dict()
for i, channel in enumerate(channels_df.chan_point.values):
    hsh, out_index = channel.split(':')
    funded_address = funding_txs[hsh]['vout'][int(out_index)]['scriptpubkey_address']        
    if funded_address not in funded_address_settlement_txs:
        funded_address_settlement_txs[funded_address] = None

In [15]:
max_reached = 0

In [16]:
# (re)run until all settlement txs are fetched
# **WARNING**, it might take some minutes
for i, funded_address in enumerate(funded_address_settlement_txs):
    if i >= max_reached:
        if not isinstance(funded_address_settlement_txs[funded_address], list) or not funded_address_settlement_txs[funded_address]:
            print(i, end='\r')
            txs = get_blockstream_address_txs(funded_address)
            if len(txs) > 2:
                # doesn't happen in our dataset
                print('Address with > 2 settlement txs:', len(txs), funded_address)
            for tx in txs:
                if funded_address in [e['prevout']['scriptpubkey_address'] for e in tx['vin']]:
                    # funded address is in inputs 
                    funded_address_settlement_txs[funded_address] = []
                    funded_address_settlement_txs[funded_address].append(tx)
        max_reached += 1

In [18]:
settlement_txs = dict()
for stxs in funded_address_settlement_txs.values():
    for stx in stxs:
        settlement_txs[stx['txid']] = stx

## 2.2 Clean Settlement Txs
Here you find the code to:
- remove txs not in GraphSense (available till `last_block`)
- remove txs involved in punishment txs

Not needed if you already read the output files above.

### 2.2.1 Remove txs not in GraphSense

In [20]:
settlement_txs = get_available_txs(settlement_txs)

### 2.2.2 Remove txs involved in punishment txs

Here you find the code to find out with settlement txs were followed by a punishment tx (details [here](https://github.com/lightningnetwork/lightning-rfc/blob/master/03-transactions.md)).

The procedure is as follows:
1. select all settlement txs that:
    - have two outputs and
    - have at least one `p2wsh` output
2. for these selected txs:
    - get the spending tx of each `p2wsh` output
    - check that the witness of the related input has length 3 and its second element is not null
    
In the following steps, additional intermediate files are generated, but we also provide them in case you don't want to run all the cells.

Not needed if you already read the output files above.

In [6]:
# intermediate files (generated in the following steps)

unilateral_settlement_txs_with_p2wsh_in_outputs_spenders = read_json(unilateral_settlement_txs_with_p2wsh_in_outputs_spenders_file)
spender_details = read_json(spender_details_file)
settlement_txs_with_punishment = read_json(settlement_txs_with_punishment_file)

In [23]:
possible_two_output_unilateral_settlement_txs = dict()
for tx in settlement_txs.values():
    if 'vout' in tx and len(tx['vout']) == 2:
        possible_two_output_unilateral_settlement_txs[tx['txid']] = settlement_txs[tx['txid']]

In [27]:
p2wsh_outputs = set() # tx_hash:out_index
unilateral_settlement_txs_with_p2wsh_in_outputs = []
for t in possible_two_output_unilateral_settlement_txs.values():
    seq = t['vin'][0]['sequence']
    if seq != 4294967295:
        # unilateral
        n_p2wsh_in_tx = 0
        for i, vout in enumerate(t['vout']):
            if vout['scriptpubkey_type'] == 'v0_p2wsh':
                p2wsh_outputs.add(t['txid'] + ':' + str(i))
                n_p2wsh_in_tx += 1
        if n_p2wsh_in_tx:
            unilateral_settlement_txs_with_p2wsh_in_outputs.append(t['txid'])

In [28]:
hash_page = dict()
for tx_hash in unilateral_settlement_txs_with_p2wsh_in_outputs:
    hash_page[tx_hash] = None

In [30]:
# **WARNING**, it might take some minutes
for i, hsh in enumerate(hash_page):
    print(i, end='\r')
    if not hash_page[hsh]:
        # better source here: https://github.com/Blockstream/esplora/blob/master/API.md#get-txtxidoutspendvout
        hash_page[hsh] = get_blockchain_page(hsh) 

In [32]:
# not needed if you have already read the intermediate files above 
unilateral_settlement_txs_with_p2wsh_in_outputs_spenders = dict()
for hsh in hash_page:
    spenders = []
    for i in [1, 2]:
        # spender = hash_page[tx_hash].split('spender')[i].split("\"")[4]
        spender = hash_page[hsh].split('spender')[i].split("\"")[6]
        if len(spender) == 64:
            spenders.append(spender)
    unilateral_settlement_txs_with_p2wsh_in_outputs_spenders[hsh] = spenders

In [57]:
# not needed if you have already read the intermediate files above
spender_details = dict()
for stx in unilateral_settlement_txs_with_p2wsh_in_outputs_spenders:
    spenders = unilateral_settlement_txs_with_p2wsh_in_outputs_spenders[stx]
    for i, spender in enumerate(spenders): # two spender txs at most
        stx_out_index = stx + ':' + str(i)
        if stx_out_index in p2wsh_outputs and spender not in spender_details:
            spender_details[spender] = None

In [60]:
# not needed if you have already read the intermediate files above
# **WARNING**, it might take some minutes
for i, tx in enumerate(spender_details):
    print(i, end='\r')
    if not spender_details[tx]:
        details = get_blockstream_tx(tx)
        spender_details[tx] = details

In [62]:
# for each spender I need not only the tx hash,
# but also the address and the amount to be more secure
spender_address_btc = set()  # spender tx hash + address + btc
for stx in unilateral_settlement_txs_with_p2wsh_in_outputs_spenders:
    spenders = unilateral_settlement_txs_with_p2wsh_in_outputs_spenders[stx]
    for i, spender in enumerate(spenders):  # two spender txs at most
        stx_out_index = stx + ':' + str(i)
        if stx_out_index in p2wsh_outputs:
            address = settlement_txs[stx]['vout'][i]['scriptpubkey_address']  # the output address of the settlement tx
            btc = settlement_txs[stx]['vout'][i]['value']
            key = spender + ':' + address + ':' + str(btc)
            spender_address_btc.add(key)

In [63]:
# the witness of the input has length 3 and the second element is not null
non_collaborative_settlement_address_btc = dict()
non_standard_witnesses = []
n_punishment_txs = 0
non_standard_witness_len = dict()
for sab in spender_address_btc:
    spender, address, btc = sab.split(':')
    details = spender_details[spender]  # cannot use spender_address_btc_details cause it has more keys
    for vin in details['vin']:
        if vin['prevout']['scriptpubkey_address'] == address and vin['prevout']['value'] == int(btc):
            len_witness = len(vin['witness'])
            if len_witness != 3:
                if len_witness not in non_standard_witness_len:
                    non_standard_witness_len[len_witness] = 0
                non_standard_witness_len[len_witness] += 1
                # print(details['txid'], 'non standard witness')
                non_standard_witnesses.append(vin['witness'])
#                 print(len(vin['witness']), end=' ')
            else:
                x = vin['witness'][1]
                if x:
                    # print(spender, x)
                    n_punishment_txs += 1
                    key = address + ':' + btc
                    if key not in non_collaborative_settlement_address_btc:
                        non_collaborative_settlement_address_btc[key] = 0
                    non_collaborative_settlement_address_btc[key] += 1


In [64]:
# not needed if you have already read the intermediate files above
settlement_txs_with_punishment = set()
# for each settlement tx
for tx in possible_two_output_unilateral_settlement_txs:
    # if one output has a p2wsh and the address and the value are in non_collaborative_settlement_address_btc
        # add settlement tx to unusable
    for vout in possible_two_output_unilateral_settlement_txs[tx]['vout']:
        if vout['scriptpubkey_type'] == 'v0_p2wsh':
            key = vout['scriptpubkey_address'] + ':' + str(vout['value'])
            if key in non_collaborative_settlement_address_btc:
                settlement_txs_with_punishment.add(tx)

In [65]:
len(settlement_txs_with_punishment)

12

In [69]:
non_punishment_funded_address_settlement_txs = dict()
for fa, stxs in funded_address_settlement_txs.items():
    non_punishment_funded_address_settlement_txs[fa] = []
    for stx in stxs:
        if stx['txid'] not in settlement_txs_with_punishment:
            non_punishment_funded_address_settlement_txs[fa].append(stx)

settlement_txs = dict()
for stxs in non_punishment_funded_address_settlement_txs.values():
    for stx in stxs:
        settlement_txs[stx['txid']] = stx
funded_address_settlement_txs = non_punishment_funded_address_settlement_txs

## 2.3 Settlement Addresses

In [16]:
settlement_addresses = set()
for fa, stxs in funded_address_settlement_txs.items():
    for tx in stxs:
        for vout in tx['vout']:
            settlement_addresses.add(vout['scriptpubkey_address'])

## 2.4 Store Data

In [15]:
write_json(unilateral_settlement_txs_with_p2wsh_in_outputs_spenders, unilateral_settlement_txs_with_p2wsh_in_outputs_spenders_file)
write_json(spender_details, spender_details_file)
write_json(list(settlement_txs_with_punishment), settlement_txs_with_punishment_file)
write_json(funded_address_settlement_txs, funded_address_settlement_txs_file)
write_json(settlement_txs, settlement_txs_file)
write_json(settlement_addresses, settlement_addresses_file)
pd.DataFrame(settlement_addresses, columns='address').to_csv(settlement_addresses_csv_file, index=False)